In [1]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
using CSV
using DataFrames
using Tables


# In this model, we test how randomly select OTUs would work in the same model. 
# This model will only work for yield_per_plant as outcome, but I would assume that
# it will also work for others with minor modification.
# I will randomly select 100 out of 2395 OTUs as predictors and feed it into the NN
# For the hidden layer, I would only add one hidden layer. As discussed in many websites
# I viewed, under very rare circumstances would a second layer improve the performance.
# Also, with 2 or more layer, the NN would be harder to train, and it is bad with the 
# small amount of data we have.
# For the number of neurons, I read about a general rule of thumb that "the optimal size 
# of the hidden layer is between the input size and the output size:, so I set the hidden 
# size to 52.
# For output, I split the values of yield_per_plant into 4 categories, based on quantile. 
# This might be problematic.

In [2]:
##### This cell import the processed data for yield_per_plant and transform the outcome data.
# It also transform the data into array type so it's easier to work with.

# Load data from the CSV file
data = CSV.read("./processed-data/otu-yield-per-plant.csv", DataFrame)
data_arr = Matrix(data)

# Split the array into otu and label
otu = data_arr[:, 2:2395]
label = data_arr[:, 2396]
# Transform the array into 4 categories based on quantile
for i in 1:length(label)
    if label[i] <= 751
        label[i] = 0
    elseif label[i] <= 1068
        label[i] = 1
    elseif label[i] <= 1444
        label[i] = 2
    else
        label[i] = 3
    end
end

In [3]:
# select 100 unique numbers between 1 to 2394 (the 50 unique predictor)
# if not unique, select again
function rand_select()
    # select 100 unique numbers between 1 to 2394 (the 50 unique predictor)
    # if not unique, select again
    rand_num = zeros(Int, 100)
    while true
        rand_num = rand((1:2394),100)
        if length(unique(rand_num)) == 100
            break
        end
    end
    return rand_num
end

rand_select (generic function with 1 method)

In [15]:
# This function divide the data into 10 part and combine the otu with labels
# and return them in a batch
# So each batch has 22 tuples of 50 otus and an encoded label
function make_fold(data, label, idx)
    # batch for otu, 100*22
    data_batch = Array{Float32, 2}(undef, length(idx), length(data[1,:]))
    for i in 1:length(idx)
        data_batch[i,:] = data[idx[i],:]
    end
    # batch for label, 1(onehot encoding)*22
    label_batch = onehotbatch(label[idx], 0:3)
    return (data_batch', label_batch)
end

make_fold (generic function with 1 method)

In [16]:
# Since we have very small amount of dataset,I am going to use 10-fold validation here
function k_fold_partition(otu_batch, label)
    # partition the whole dataset into 10 folds
    fold_idx = partition(1:length(otu_batch[:,1]), length(otu_batch[:,1])÷10+1)
    # call make_fold and store the 10 folds
    whole_set = [make_fold(otu_batch, label, i) for i in fold_idx]

    return whole_set
end


k_fold_partition (generic function with 1 method)

In [17]:
# model construction
@info("Feed-Forward NN construction")
model = Chain(
    # Input 100 predictors and feed into 52 neurons in the hidden layer
    # use ReLu as activation function
    Dense(100, 52, relu),
    # feed 27 neurons to the output, which consists of 4 categories
    Dense(52, 4),
    # use softmax as the activation function
    softmax
)

┌ Info: Feed-Forward NN construction
└ @ Main In[17]:2


Chain(Dense(100, 52, relu), Dense(52, 4), softmax)

In [18]:
# Train the model in this cell
function train(whole_set)
    # record the accuracy for each folder
    mean_accuracy = zeros(Float32,10)
    # loop for 10 folds
    for k in 1:10
        # set the kth folder as testing set
        train_set = gpu(whole_set[Not(k)])
        test_set = gpu(whole_set[k])
        best_acc = 0.0
        # reset all the parameters
        Flux.loadparams!(model, map(p -> p .= randn.(), Flux.params(model)))
        # set to terminate the epoches if not improved for too long
        last_improvement = 0
        # crossentropy as loss function
        loss(x, y) = crossentropy(model(x),y)
        # take mean of accuracy
        accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
        # when learning rate is larger(0.1), the accuracy is far worse and some time result in the
        # same value with different input
        # when it's smaller(0.001), the result does not differ much
        opt = ADAM(0.01)
        # 100 epoches for each folder
        for epoch_idx in 1:200
            Flux.train!(loss, params(model), train_set, opt)
            acc = accuracy(test_set...)

            if acc >= 0.999
                best_acc = acc
                break
            end
            # update the best accuracy and last improvement
            if acc >= best_acc
                best_acc = acc
                last_improvement = epoch_idx
            end
            # no improvement for too long
            if epoch_idx - last_improvement >= 20 && opt.eta > 1e-6
                opt.eta /= 10.0
                # After dropping learning rate, give it a few epochs to improve
                last_improvement = epoch_idx
            end
            if epoch_idx - last_improvement >= 50
                break
            end
    
        end
        # save the best accuracy for this folder
        mean_accuracy[k] = best_acc
    end
    # output the average accuracy for 10 folders
    return mean(mean_accuracy)
end


train (generic function with 1 method)

In [19]:
function batch_info(batch) 
    zero = zeros(Float32,100)
    m = zeros(Float32,100)
    v = zeros(Float32,100)
    for i in 1:length(batch[1,:])
        count = 0
        for j in 1:length(batch[:,1])
            if batch[j,i] != 0
                count += 1
            end
        end
        zero[i] = count
        m[i] = mean(batch[:,i])
        v[i] = var(batch[:,i])
    end
    return (mean(zero),mean(m), mean(v))
end

batch_info (generic function with 1 method)

In [20]:
# randomly select 100 otus and run the whole procedure, record information
# for general_info, each row is a selection
# for columns, 
# 1-100: otu indices for the selection 
# 101: number of non-zero values
# 102: weighted mean for the 100 otus
# 103: weighted variance for the 100 otus
# 104: average accuracy(10 folds) for this selection 
general_info = Array{Float32}(undef, 100, 104)
for t in 1:length(general_info[:,1])
    # randomly select 100 number from 1 to 2049
    rand_num = rand_select()
    # select the corresponding OTUs
    otu_batch = otu[:,rand_num]
    a = Flux.Data.DataLoader(otu_batch, batchsize=22)

    # split the dataset into 10 folds
    whole_set = k_fold_partition(otu_batch, label)
    
    accuracy = train(whole_set)
    info = batch_info(otu_batch)
    
    general_info[t,1:100] .= rand_num
    general_info[t,101:103] .= info
    general_info[t,104] = accuracy

end


In [21]:
# Merge_sort as I don't know how to use the build-in sort here

# A recursion function, break the array in to parts and merge with comparison
function merge_sort(arr, left, right) 
    # terminate condition
    if right > left 
        # get the mid point to break the array into 2 parts
        mid = div(left+right, 2)
        # recurse on the left
        merge_sort(arr, left, mid)
        # recurse on the right
        merge_sort(arr, mid+1, right)
        # merge the array by comparison
        merge(arr, left, mid, right)
    end
end

# Merge function to sort the input in order
function merge(arr, left, mid, right)
    # temp array to store sorted index, from start point to end point
    temp = zeros(right-left+1,104)
    i = left
    j = mid+1
    k = 1
    
    # compare both part of the input index by index, w.r.t the mid point
    # store the smaller index into the temp array and increment that index
    while i <= mid && j <= right
        if arr[i,104] <= arr[j,104]
            temp[k,:] = arr[i,:]
            k += 1
            i += 1
        else 
            temp[k,:] = arr[j,:]
            k += 1
            j += 1
        end
    end
    
    # after the comparison, copy whatever's left in both part
    while i <= mid
        temp[k,:] = arr[i,:]
        k += 1
        i += 1
    end
    
    while j <= right
        temp[k,:] = arr[j,:]
        k += 1
        j += 1
    end
    
    # copy everything in the temp array into the original
    for m in left:right
        arr[m,:] = temp[m-left + 1,:]
    end
    
    return arr
end

merge (generic function with 1 method)

In [22]:
# sort the info with regard to accuracy and write them into a CSV
sorted = merge_sort(general_info, 1, length(general_info[:,104]))
CSV.write("./processed-data/model-info-yp.csv", Tables.table(sorted), header=false)

"./processed-data/model-info-yp.csv"